In [ ]:
pip install clawpack

In [ ]:
# @title Euler 1D set 2
from clawpack import pyclaw
from clawpack import riemann
from funciones import funcion_paso
# Importar el solucionador
solver = pyclaw.ClawSolver1D(riemann.euler_1D_py.euler_roe_1D)
solver.kernel_language = "Python"
# Condiciones de frontera.
solver.bc_upper[0] = pyclaw.BC.extrap
solver.bc_lower[0] = pyclaw.BC.extrap
# Dominio
domain = pyclaw.Domain([0], [10], [500])
# Objeto de solución
solution = pyclaw.Solution(solver.num_eqn, domain)
# Condición inicial
state = solution.state
# Centros de celdas de dominio espacial
xc = state.grid.p_centers[0]
from numpy import exp
# Gamma
cda_gamma = 1.4
# Definicion de funciones auxiliares de
# variables independientes y conservadas
densidad = funcion_paso(xc-0.01, 5, 3, 1)
velocidad = 0.9
presion = funcion_paso(xc-0.01, 5, 3, 1)
energia = presion/(cda_gamma - 1) + 0.5*densidad*velocidad**2
# Asignación de variables conservadas
state.q[0,:] = densidad
state.q[1,:] = densidad*velocidad
state.q[2,:] = energia
# Parámetros
state.problem_data["gamma"] = cda_gamma
state.problem_data["gamma1"] = cda_gamma - 1
state.problem_data["efix"] = False
# Controlador
controller = pyclaw.Controller()
controller.solution = solution
controller.solver = solver
# Veces que se imprimen los datos
controller.num_output_times = 5
# Tiempo final de simulación
controller.tfinal = 2
controller.output_format = "ascii"
controller.outdir = "set3"
# Correr la simulación
status = controller.run()

In [ ]:
# @title Euler 1D set 1
nombre = "set1"
from clawpack import pyclaw
from clawpack import riemann
from funciones import funcion_paso
# Importar el solucionador
solver = pyclaw.ClawSolver1D(riemann.euler_1D_py.euler_roe_1D)
solver.kernel_language = "Python"
# Condiciones de frontera.
solver.bc_upper[0] = pyclaw.BC.extrap
solver.bc_lower[0] = pyclaw.BC.extrap
# Dominio
domain = pyclaw.Domain([0], [10], [500])
# Objeto de solución
solution = pyclaw.Solution(solver.num_eqn, domain)
# Condición inicial
state = solution.state
# Centros de celdas de dominio espacial
xc = state.grid.p_centers[0]
from numpy import exp
# Gamma
cda_gamma = 1.4
# Definicion de funciones auxiliares de
# variables independientes y conservadas
densidad = 1 + exp(-((xc-0.01)-5)**2)
velocidad = 1.0
presion = 0.5
energia = presion/(cda_gamma - 1) + 0.5*densidad*velocidad**2
# Asignación de variables conservadas
state.q[0,:] = densidad
state.q[1,:] = densidad*velocidad
state.q[2,:] = energia
# Parámetros
state.problem_data["gamma"] = cda_gamma
state.problem_data["gamma1"] = cda_gamma - 1
state.problem_data["efix"] = False
# Controlador
controller = pyclaw.Controller()
controller.solution = solution
controller.solver = solver
controller.num_output_times = 5
controller.tfinal = 3
controller.output_format = "ascii"
controller.outdir = nombre
# Correr la simulación
status = controller.run()

In [ ]:
import pandas as pd
def contar_espacios(cadena):
    contador_espacios = 0

    for caracter in cadena:
        if caracter == ' ':
            contador_espacios += 1

    return contador_espacios

def separar(cadena: str):
  valores = cadena.split(" ")
  valores = [valor for valor in valores if valor]
  return valores

def formatear(file_path: str, header = 4) -> pd.DataFrame:
  datos = pd.read_csv(file_path, header = header)
  datos.columns = ["valores"]

  datos_separados = datos["valores"].apply(func=separar)
  df = pd.DataFrame(data={})
  df[["q0", "q1", "q2"]] = datos_separados.apply(pd.Series).astype(float)
  return df

def calcular_variables(datos: pd.DataFrame) -> pd.DataFrame:
  datos["densidad"] = datos["q0"]
  datos["velocidad"] = datos["q1"] / datos["q0"]
  datos["presion"] = (cda_gamma-1)*(datos["q2"] - 0.5*datos["densidad"]*datos["velocidad"]**2)

  return datos


In [ ]:
datos_t0 = formatear("set2/fort.q0000", header=4)

datos_tf = formatear("set2/fort.q0005", header=4)

datos_t0 = calcular_variables(datos_t0)
datos_tf = calcular_variables(datos_tf)

import matplotlib.pyplot as plt
import numpy as np
x_dom = np.linspace(0,9.98,500)

plt.plot(x_dom, datos_tf['velocidad'].values)
plt.ylim(-0.05, 0.5)
plt.show()


In [ ]:
import os
def obtener_tiempo(id: str, folder: str):
  # Path del archivo donde está el tiempo
  path = folder + "/" + "fort.t" + id

  with open(path, "r") as archivo:
    # Leer la primera línea del archivo
    contenido = archivo.read().split("\n")[0]
    # Obtener el tiempo, buscandolo separado entre espacios en la primera línea
    tiempo = [x for x in contenido.split(" ") if x][0]
    # Convertir el tiempo a flotante
    tiempo = float(tiempo)

  return tiempo


# Ruta de la carpeta
carpeta = "set3"
# Obtener la lista de archivos en la carpeta
archivos = os.listdir(carpeta)
# Obtener los ids de cada archivo en un dic, ssi son archivos de datos
ids_waves = {x:x[-5:] for x in archivos if x[-5] in ("q")}
ids_time = {x:x[-5:] for x in archivos if x[-5] in ("t")}

solution_data = pd.DataFrame()

for archivo in ids_waves.keys():
  # Obtener correlativo de impresión de datos
  id_num_out = ids_waves[archivo][1:]
  # Obtener tiempo
  tiempo = obtener_tiempo(id_num_out, carpeta)
  # Elaborar ruta al archivo de datos
  ruta_a_archivo = carpeta + "/" + "fort.q" + id_num_out
  # Generar dataframe con datos de la solución de un instante
  df_to_concat = formatear(ruta_a_archivo, header=4)
  # Calcular variables físicas independientes
  df_to_concat = calcular_variables(df_to_concat)
  # Asignar tiempo
  df_to_concat["t"] = tiempo
  df_to_concat["x"] = np.linspace(0,9.98,500)
  # Concatenar los datos
  if solution_data.shape[0] == 0:
    solution_data = df_to_concat
  else:
    solution_data = pd.concat([solution_data, df_to_concat])

solution_data = solution_data.reset_index().sort_values(by=["t", "x"])

solution_data.to_csv(f"data_solucion_{carpeta}.csv", sep="\t", index=False)
solution_data["t"].value_counts()

In [ ]:
import numpy as np

import numpy as np

def funcion_paso(x: np.array, cutoff: float, valor1: float, valor2: float) -> np.array:
    """
    Implementa una función escalón en una matriz NumPy.

    Parámetros:
    - x (np.array): Matriz NumPy de entrada.
    - cutoff (float): Valor de corte que determina la transición entre los dos valores de la función escalón.
    - valor1 (float): Valor constante para los elementos en x que son menores que cutoff.
    - valor2 (float): Valor constante para los elementos en x que son mayores o iguales a cutoff.

    Retorna:
    - np.array: Matriz NumPy resultante después de aplicar la función escalón.

    Ejemplo:
    >>> x = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
    >>> funcion_paso(x, 3.0, 10.0, 20.0)
    array([10., 10., 20., 20., 20.])
    """
    condiciones = [x <= cutoff, x > cutoff]
    valores = [valor1, valor2]
    result = np.piecewise(x, condiciones, valores)
    return result


# Ejemplo de uso
x_values = np.linspace(-1,10,11)

a = funcion_paso(x_values, 0, -4, 5)
a